In [1]:
import os

In [2]:
os.chdir("../")

In [18]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class PrepareBaseModelConfig:
     root_dir:Path
     base_model_path:Path
     updated_model_path:Path
     params_image_size: list
     params_learning_rate: float
     params_include_top : bool
     params_weights: str
     params_classes: int


In [4]:
from src.ChickenDisease.constants import *
from src.ChickenDisease.utils.common import read_yaml,create_directories

In [5]:
class ConfigurationManger:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            param_filepath= PARAM_FILE_PATH):
            
            self.config= read_yaml(config_filepath)
            self.param= read_yaml(param_filepath)

            create_directories([self.config.artifacts_root])
        
    
    def get_prepare_base_model_config(self)-> PrepareBaseModelConfig:
          config=self.config.prepare_base_model
          create_directories([config.root_dir])

          prepare_base_model_config= PrepareBaseModelConfig(
                root_dir=Path(config.root_dir),
                base_model_path=Path(config.base_model_path),
                updated_model_path=Path(config.updated_model_path),
                params_image_size=self.param.IMAGE_SIZE,
                params_learning_rate=self.param.LEARNING_RATE,
                params_include_top=self.param.INCLUDE_TOP,
                params_weights=self.param.WEIGHTS,
                params_classes=self.param.CLASSES,
                )

          return prepare_base_model_config
    

In [6]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model


In [19]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config=config

    
    def get_base_model(self):
        self.base_model=tf.keras.applications.EfficientNetB7(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        self.save_model(path=self.config.base_model_path, model=self.base_model)

    @staticmethod
    def prepare_full_model(model,classes,learning_rate):

        model.trainable= True
        x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(model.output)
        x = Dense(1024, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
        x=Dropout(rate=.3, seed=123)(x)
        x = Dense(128, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                        bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
        x=Dropout(rate=.45, seed=123)(x)        
        output=Dense(classes, activation='softmax')(x)
        model=Model(inputs=model.input, outputs=output)
        
        full_model=tf.keras.models.Model(
            inputs=model.input,
            outputs=output
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adamax(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
            metrics=['accuracy']
        )

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model= self.prepare_full_model(
            model=self.base_model,
            classes=self.config.params_classes,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_model_path, model=self.full_model)

    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)
    


In [20]:
try:
    config= ConfigurationManger()
    prepare_base_model_config= config.get_prepare_base_model_config()
    prepare_base_model=PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2023-09-18 11:50:52,158: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-18 11:50:52,161: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-18 11:50:52,162: INFO: common: created directory at: artifacts]
[2023-09-18 11:50:52,163: INFO: common: created directory at: artifacts/prepare_base_model]
[2023-09-18 11:50:59,313: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_10 (Rescaling)    (None, 224, 224, 